In [191]:
import sys
import argparse
import warnings
from pathlib import Path
from scripts.folder_structure import FolderStructure
from scripts.input_processing import InputProcessor
from scripts.variables_manager import VariablesManager
from scripts.mmseqs_clustering import MMseqsClustering
from scripts.gwas import GWASWorkflow
from scripts.processing import Processor

%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 40)

# workflow IO
input_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT"
output_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-02-12_KLEBDATA_LIGHT_GWAS"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [192]:
# folder structure
folder_manager = FolderStructure(input_dir, output_dir)
structure = folder_manager.get_paths_dict()
params = folder_manager.get_params_dict()

Created directory: /Users/januszkoszucki/tmp_blast_dir


In [193]:
# input processor
inprocessor = InputProcessor(structure, params)
inprocessor.process_bacteria_table()
inprocessor.process_bacteria_iqtree()
inprocessor.process_prophage_table()
inprocessor.process_prophage_proteins()
inprocessor.process_function_predictions(run=True)
inprocessor.process_recombinant_depos(run=False)


Processed bacteria table saved.
Bacterial genomes (n=3911): 2527 retained (-1384, where 10 have more than 500 contigs and 1383 not in KPN KVV KQQ KQS)

Reading not filtered table of prophages!
Processed prophages table saved.
Prophage genomes (n=8105): 8103 retained (-2 that were found in excluded genomes)

Reading already filtered prophages by confidence and completeness! 
Prophage protein files (n=8105): retained 8103 (-2 that were found in excluded genomes)
Prophage proteins processed and saved. [proteins: 105551 with length >= 300] [removed: 371405]

Duplicated genes at the end of phages are removed here! A problem of mgg_annotation pipeline.


In [181]:
# variables manager
manager = VariablesManager(structure, params)
mmseqs_vars    = manager.get_mmseqs_vars()
proc_clus_vars = manager.get_process_clustering_vars()
matrix_vars    = manager.get_matrix_vars()
functions_vars = manager.get_map_functions_vars()

lasso_vars        = manager.get_gwas_vars(mode='lasso')
elastic_net_vars  = manager.get_gwas_vars(mode='elastic_net')

In [182]:
# mmseqs clustering
clustering = MMseqsClustering(*mmseqs_vars)
clustering.run_mmseqs(run=False)
clustering.clean_clustering()
clustering.process_clustering(*proc_clus_vars)
clustering.compute_matrix(*matrix_vars)
clustering.map_functions(*functions_vars)
clustering.get_alignments()

In [11]:
# gwas
gwas = GWASWorkflow(structure, params)
gwas.get_input_files()
gwas.compute_n_variants(run=False)
gwas.get_script(*lasso_vars)
gwas.get_script(*elastic_net_vars)
gwas.run_scripts()

37 phenotypes 20x bootstrap with replacement [sample size = 2478] [genomes without prophages excluded]
KL1.. KL2.. KL3.. KL6.. KL7.. KL8.. KL10.. KL11.. KL13.. KL14.. KL15.. KL16.. KL21.. KL22.. KL24.. KL25.. KL28.. KL30.. KL35.. KL38.. KL39.. KL47.. KL48.. KL55.. KL57.. KL60.. KL61.. KL62.. KL64.. KL103.. KL111.. KL122.. KL125.. KL127.. KL142.. KL146.. KL153.. 
Done!
Number of variants per locus exist... remove existing file to run again. 
Running lasso... Done!
Running elastic net... Done!


In [12]:
# processor
processor = Processor(structure, params)
processor.concatenate_pyseer()
processor.compute_metrics(run=True)
processor.map_functions_topologies()
processor.combine_info_bootstrap()
processor.pyseer_hits_with_CI()

WARNING! No pyseer results found for mode incorrect!
Metrics for variants exist... remove existing files to run again. 
Pyseer hits with statistics (CI) already computed... remove existing file to run again... loading existing file.
